# Deep learning volume segmentation

Authors: Alessia Saccardo (s212246@dtu.dk) & Felipe Delestro (fima@dtu.dk)

This notebook aims to demonstrate the feasibility of implementing a comprehensive deep learning segmentation pipeline solely leveraging the capabilities offered by the qim3d library. Specifically, it will highlight the utilization of the annotation tool and walk through the process of creating and training a Unet model.

### Imports

In [ ]:
import qim3d
import numpy as np 
import os 

### Load data
Qim3d library contains a set of example volumes which can be easily loaded using `qim3d.examples.{volume_name}`

In [ ]:
vol = qim3d.examples.bone_128x128x128

To easily have an insight of how the volume looks like we can interact with it using the `slicer` function from `qim3d`

In [ ]:
qim3d.viz.slicer(vol)

# Generate dataset for training

In order to train the classification model, we need to create a dataset from the volume.

This means that we'll need a few slices to be used for `training` and at least one for the `test`

The dataset for training the model is managed by `qim3d.utils.prepare_datasets` and it expects files to follow this structure:

<pre>
dataset
├── test
│   ├── images
│   │   └── FileA.png
│   └── labels
│       └── FileA.png
└── train
    ├── images
    │   ├── FileB.png
    │   └── FileC.png
    └── labels
        ├── FileB.png
        └── FileC.png


</pre>


In [ ]:
# Number of slices that will be used
ntraining = 4
ntest = 1

In the following cell, we get the slice indices, making sure that we're not using the same indices for training and test.

In [ ]:
# Create a set with all the indices
all_idxs = set(range(vol.shape[0]))

# Get indices for training data
training_idxs = list(np.random.choice(list(all_idxs), size=ntraining))
print(f"Slices for training data...: {training_idxs}")

# Get indices for test data
test_idxs = list(np.random.choice(list(all_idxs - set(training_idxs)), size=ntest, replace=False))
print(f"Slices for test data.......: {test_idxs}")

## Create folder structure
Here we create the necessary directories

In [ ]:
# Base path for the training data
base_path = os.path.expanduser("~/dataset")

# Create directories
print("Creating directories:")
for folder_split in ["train", "test"]:
    for folder_type in ["images", "labels"]:
        path = os.path.join(base_path, folder_split, folder_type)
        os.makedirs(path, exist_ok=True)
        print(path)

# Here we have the option to remove any previous files
clean_files = True
if clean_files:
    for root, dirs, files in os.walk(base_path):
        for file in files:
            file_path = os.path.join(root, file)
            os.remove(file_path)

# Annotate data

The following cell will generate an annotation tool for each slice that was requested. 

You should use the tool to drawn a mask over the structures you're willing to detect, and press the button `Update` so that the mask is saved.

In [ ]:
annotation_tools = {}
for idx in training_idxs + test_idxs:
    if idx in training_idxs:
        subset = "training"
    elif idx in test_idxs:
        subset = "test"
    annotation_tools[idx] = qim3d.gui.annotation_tool.Interface()
    annotation_tools[idx].name_suffix = f"_{idx}"  
    print(f"Annotation for slice {idx} ({subset})")
    annotation_tools[idx].launch(vol[idx])

### Getting masks from the annotation tool
The masks are stored in the annotation tool when the button `Update` is pressed

Here we extract the masks and save them to disk, followign the standard needed for the DL model

In [ ]:
print("Saving images and masks to disk")
for idx in training_idxs + test_idxs:
    
    if idx in training_idxs:
        folder_split = "train"

    elif idx in test_idxs:
        folder_split = "test"

    print (f"- slice {idx} ({folder_split})")
    mask_dict = annotation_tools[idx].get_result()
    mask = list(mask_dict.values())[0]

    # Save image
    qim3d.io.save(os.path.join(base_path,folder_split,"images",f"{idx}.png"), vol[idx], replace=True)
    # Save label
    qim3d.io.save(os.path.join(base_path,folder_split,"labels",f"{idx}.png"), mask, replace=True)


# Build Unet

Building and training the Unet model is straightforward using `qim3d`. 

We first need to instantiate the model by defying its size, which can be either *small*, *medium* or *large*. 

In [ ]:
# defining model
model = qim3d.models.UNet(size = 'small', dropout = 0.25)

Then we need to decide which type of augumentation to apply to the data. 

The `qim3d.utils.Augmentation` allows to specify how the images should be reshaped to the appropriate size and the level of transformation to apply respectively to train, test and validation sets. 

The resize must be choosen between [*crop*, *reshape*, *padding*] and the level of transformation must be chosse between [*None*, *light*, *moderate*, *heavy*]. The user can also specify the mean and standard deviation values for normalizing pixel intensities.

In [ ]:
# defining augmentation
aug = qim3d.utils.Augmentation(resize = 'crop', transform_train = 'light')

Then the datasets and dataloaders are instantiated 

In [ ]:
# datasets and dataloaders
train_set, val_set, test_set = qim3d.utils.prepare_datasets(path = base_path,
                                                            val_fraction = 0.5,
                                                            model = model,
                                                            augmentation = aug)


train_loader, val_loader, test_loader = qim3d.utils.prepare_dataloaders(train_set, 
                                                                        val_set,
                                                                        test_set,
                                                                        batch_size = 1)

The hyperparameters are defined using the function `qim3d.models.Hyperparameters` and the model can be easily trained by running the function `qim3d.utils.train_model` which returns also a plot of the losses at the end of the training if the option is selected by the user 

# Train model

In [ ]:
# model hyperparameters
hyperparameters = qim3d.models.Hyperparameters(model, n_epochs=10, 
                                               learning_rate = 5e-3, loss_function='DiceCE',
                                               weight_decay=1e-3)

# training model
qim3d.utils.train_model(model, hyperparameters, train_loader, val_loader, plot=True)

# Check results

To compute the inference step it is just needed to run `qim3d.utils.inference`.

The results can be visualize with the function `qim3d.viz.grid_pred` that shows the predicted segmentation along with a comparison between the ground truth.

In [ ]:
in_targ_preds_test = qim3d.utils.inference(test_set, model)
qim3d.viz.grid_pred(in_targ_preds_test,alpha=1)

# Compute inference on entire volume

Given that the input is a volume, the goal is to perform inference on the entire volume rather than individual slices.

By using the function `qim3d.utils.volume_inference` it is possible to obtain the segmentation volume output

In [ ]:
inference_vol = qim3d.utils.models.volume_inference(vol, model)
qim3d.viz.slicer(inference_vol)

We can also visualize the created mask together with the original volume

In [ ]:
vol_masked = qim3d.viz.vol_masked(vol, inference_vol, viz_delta=128)
qim3d.viz.slicer(vol_masked, cmap="PiYG")